# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [5]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print (df.head())

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [45]:
# frequency table for prestige and whether or not someone was admitted

#  counts  = df.value_counts()   showed error: AttributeError: 'DataFrame' object has no attribute 'value_counts'.
# Not sure why that was the error shown


my_freq_table = pd.crosstab(index = df["admit"], columns = df["prestige"])
my_freq_table.index  = ["admitted", "not admitted"]
my_freq_table

prestige,1.0,2.0,3.0,4.0
admitted,28,95,93,55
not admitted,33,53,28,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [109]:
# CREATING CLASS VARIABLES    -- not sure whether corect 

def prestige_classifier(row):
    if row['prestige'] == 1:
        return 1
    elif row['prestige'] == 2:
        return 2
    elif row['prestige'] == 3:
        return 3
    elif row['prestige'] == 4:
        return 4

prestige_classifier

df_with_classifier = pd.concat([df, prestige_classifier], axis = 1)  # error shown, do not know why cannot concetanate a class vaiable to a table 
df_with_classifier.drop(["prestige" ], inplace=True, axis =1 )
df_with_classifier.head()

TypeError: cannot concatenate a non-NDFrame object

In [66]:
# CREATING DUMMY VARIABLES

#  for elem in df["prestige"].unique():
#    df[str(elem)] = df["prestige"] == elem
# above code does not work, not sure why 

df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
prestige_dummies = pd.get_dummies(df["prestige"]).rename(columns = lambda x:'Prestige_'+str(int(x)))

step1 = pd.concat([df, prestige_dummies], axis = 1)

# step1.drop(["prestige", '1.0' ], inplace=True, axis =1 )  
# this code above does not work, error:  labels ['1.0'] not contained in axis

step1.drop(["prestige" ], inplace=True, axis =1 )
df.head()
step1.head()

,admit,gre,gpa,Prestige_1,Prestige_2,Prestige_3,Prestige_4
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


### 2.2 When modeling our class variables, how many do we need? 



Answer: Should be 3 class variables, as there are total 4 prestige options: 1.0, 2.0,3.0, 4.0. 
Not sure how to drop 


## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [68]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(prestige_dummies.ix[:, 'Prestige_1':])    # what does this attribute mean: .ix ?
print (handCalc.head())

   admit    gre   gpa  Prestige_1  Prestige_2  Prestige_3  Prestige_4
0      0  380.0  3.61           0           0           1           0
1      1  660.0  3.67           0           0           1           0
2      1  800.0  4.00           1           0           0           0
3      1  640.0  3.19           0           0           0           1
4      0  520.0  2.93           0           0           0           1


In [70]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted


my_freq_table_prestige1 = pd.crosstab(index = step1["admit"], columns = step1["Prestige_1"])
my_freq_table_prestige1.index  = ["admitted", "not admitted"]
my_freq_table_prestige1


Prestige_1,0,1
admitted,243,28
not admitted,93,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [74]:
print (28 / 33)

0.8484848484848485


In [75]:
import scipy 
oddsratio, pvalue = scipy.stats.fisher_exact([[243,28], [93,33]])
print(pvalue)
print(oddsratio)

8.49263064244e-05
3.07949308756


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [77]:
print (243 / 93)

2.6129032258064515


#### 3.3 Calculate the odds ratio

In [78]:
print (0.84 / 2.6129032258064515)

0.3214814814814815


#### 3.4 Write this finding in a sentenance: 

Answer: The odds of being admitted when a person had attended a prestige 1 school is:  32.%

#### 3.5 Print the cross tab for prestige_4

In [59]:

my_freq_table_prestige4 = pd.crosstab(index = step1["admit"], columns = step1["Prestige_4.0"])
my_freq_table_prestige4.index  = ["admitted", "not admitted"]
my_freq_table_prestige4

Prestige_4.0,0,1
admitted,216,55
not admitted,114,12


#### 3.6 Calculate the OR 

In [79]:
print (55/12)

4.583333333333333


#### 3.7 Write this finding in a sentence

Answer: The odds of being admitted when the person attended a Prestige 4 college is 4.5


## Part 4. Analysis

In [94]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(prestige_dummies.ix[:, 'prestige_2':])
print(data.head())

   admit    gre   gpa
0      0  380.0  3.61
1      1  660.0  3.67
2      1  800.0  4.00
3      1  640.0  3.19
4      0  520.0  2.93


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [96]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [97]:
data.head()


,admit,gre,gpa,intercept
0,0,380.0,3.61,1.0
1,1,660.0,3.67,1.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,1.0
4,0,520.0,2.93,1.0


In [101]:
train_cols  = cols_to_keep

In [102]:
data['admit']

0      0
1      1
2      1
3      1
4      0
5      1
6      1
7      0
8      1
9      0
10     0
11     0
12     1
13     0
14     1
15     0
16     0
17     0
18     0
19     1
20     0
21     1
22     0
23     0
24     1
25     1
26     1
27     1
28     1
29     0
      ..
370    1
371    1
372    1
373    1
374    0
375    0
376    0
377    1
378    0
379    0
380    0
381    1
382    0
383    0
384    1
385    0
386    1
387    0
388    0
389    0
390    1
391    1
392    1
393    1
394    1
395    0
396    0
397    0
398    0
399    0
Name: admit, Length: 397, dtype: int64

#### 4.2 Fit the model

In [100]:

from sklearn import datasets, neighbors, metrics, model_selection

knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform')  # to use the 5 nearest neighbours by voting 


knn.fit(train_cols, data['admit'])   # this is the code to train the model. then the model will have certain parameters
print (knn.predict(train_cols))


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Found input variables with inconsistent numbers of samples: [1, 397]

#### 4.3 Print the summary results

In [110]:
knn.summary()

AttributeError: 'KNeighborsClassifier' object has no attribute 'summary'

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [111]:
np.exp(X)

NameError: name 'X' is not defined

In [ ]:
conf['OR'] = params

conf.columns = ['2.5%', '97.5%', 'OR']


#### 4.5 Interpret the OR of Prestige_2

Answer: 

#### 4.6 Interpret the OR of GPA

Answer: 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [128]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n // arrays[0].size                   # have to use double /,  "//
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [129]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print (gres)
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print (gpas)
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.0]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [132]:
# CREATING DUMMY VARIABLES

## to be honest, do not know which dummy variables to create. As there are now only discrete numerical values in [1, 2, 3, 4], and float number values in gpa and gre
# where are the discrete variables so i can create dummy variables from?

df_New =combos.dropna() 
combos_dummies = pd.get_dummies(combos[[1, 2, 3, 4]]).rename(columns = lambda x:'Prestige_'+str(int(x)))

step2 = pd.concat([combos, combos_dummies], axis = 1)

step2.drop([[1, 2, 3, 4]], inplace=True, axis =1 )

step2.head()

KeyError: '[4] not in index'

#### 5.2 Make predictions on the enumerated dataset

In [134]:
predictions = combos.apply(my_classifier, axis=1)
combos['predictions'] = predictions
print (float(len(combos[combos.target == combos.predictions])) / len(combos))

NameError: name 'my_classifier' is not defined

#### 5.3 Interpret findings for the last 4 observations

Answer:  If the code works, we should be able to draw conclusions on how GRE, GPA and prestige increase probability of admission 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.

In [137]:
k = range(2, 10)
params = {'n_neighbors': k }
gv = GridSearchCV(
    estimator = KNeighborsClassifier(),
    param_grid = params,
    cv = kf
)
gv.fit(X, y)

# not sure what is the meaning of stratified
# only have an idea on how to plot GRE  and GPA 

plt.plot(k, combos['GPA', 'GRE'])

NameError: name 'GridSearchCV' is not defined